In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(8, 8*8, 3, stride=2)
        self.relu2 = nn.ReLU()

        self.head = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(256, 128), 
            nn.Dropout(0.2),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        y = self.head(x)
        y = self.dequant(y)
        return y
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            # print(correct)
            # print((len(y) * 100))
            # print(correct * 100. / (len(y) * 100))
            correct = 0


def test(dataloader, model, loss_fn, cuda=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            if cuda:
                X, y = X.to('cuda'), y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)

model_fp32_fused = torch.ao.quantization.fuse_modules(model,
    [['conv1', 'relu1'], ['conv2', 'relu2']])

model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused.train(), inplace=False)
optimizer = torch.optim.SGD(model_fp32_prepared.parameters(), lr=1e-3)

for i in range(100):
    model_fp32_prepared = model_fp32_prepared.to('cuda')
    train(train_dataloader, model_fp32_prepared, loss_fn, optimizer)
    model_fp32_prepared = model_fp32_prepared.to('cpu')
    test(test_dataloader, model_fp32_prepared, loss_fn)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/ReduceAllOps.cpp:71.)
  return torch.fused_moving_avg_obs_fake_quant(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:687.)
  retur

Test Error: 
 Accuracy: 10.0%, Avg loss: 2.300668 

Test Error: 
 Accuracy: 21.1%, Avg loss: 2.294422 

Test Error: 
 Accuracy: 13.1%, Avg loss: 2.287112 

Test Error: 
 Accuracy: 12.0%, Avg loss: 2.276328 

Test Error: 
 Accuracy: 13.4%, Avg loss: 2.259450 

Test Error: 
 Accuracy: 15.9%, Avg loss: 2.231156 

Test Error: 
 Accuracy: 20.2%, Avg loss: 2.183709 

Test Error: 
 Accuracy: 27.3%, Avg loss: 2.103549 

Test Error: 
 Accuracy: 36.5%, Avg loss: 1.964199 

Test Error: 
 Accuracy: 48.6%, Avg loss: 1.751757 

Test Error: 
 Accuracy: 54.8%, Avg loss: 1.533533 

Test Error: 
 Accuracy: 58.7%, Avg loss: 1.372290 

Test Error: 
 Accuracy: 60.2%, Avg loss: 1.266324 

Test Error: 
 Accuracy: 61.2%, Avg loss: 1.197055 

Test Error: 
 Accuracy: 61.7%, Avg loss: 1.151290 

Test Error: 
 Accuracy: 62.2%, Avg loss: 1.118056 

Test Error: 
 Accuracy: 62.3%, Avg loss: 1.096168 

Test Error: 
 Accuracy: 62.9%, Avg loss: 1.075209 

Test Error: 
 Accuracy: 63.4%, Avg loss: 1.061473 

Test Error: 

In [5]:
model_fp32_prepared.to("cpu")
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared, inplace=False)

In [6]:
%%timeit
test(test_dataloader, model_fp32_prepared, loss_fn)

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.669691 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669874 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669874 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669876 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669743 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669756 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669754 

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.669751 

1.26 s ± 6.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
test(test_dataloader, model_int8, loss_fn)

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

Test Error: 
 Accuracy: 75.0%, Avg loss: 0.670549 

937 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
torch.save(model_int8, "model_int8_qat.pt")